# Frequent Itemset Mining

### Student: Rodolfo Lerma

Learning Objectives:

- Extract frequent patterns given a corpus of data.
- Find the rules which are interesting and non-obvious for a given domain.

In [1]:
import pandas as pd
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [2]:
df = pd.read_excel('Online Retail.xlsx')
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [3]:
df.shape

(541909, 8)

In [4]:
columns = df.columns.to_list()

In [5]:
df2 = pd.read_excel('online_retail_II.xlsx')
df2.rename({'Invoice':'InvoiceNo', }, axis = 1, inplace = True)
df2.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


In [6]:
df3 = df2.set_axis(columns, axis=1, inplace=False)

In [7]:
df3.shape

(525461, 8)

### Question 1.1: Concatinate both dataframes to create a single dataframe. Remove any rows where InvoiceNo is Null and Quantity is Negative

In [8]:
frames = [df, df3]
data = pd.concat(frames, axis = 0, sort=False)

In [9]:
data.head(2)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [10]:
data.shape

(1067370, 8)

It is possible to see that the number of rows and columns match what we had before in the previous 2 datasets.

In [11]:
data.dtypes

InvoiceNo              object
StockCode              object
Description            object
Quantity                int64
InvoiceDate    datetime64[ns]
UnitPrice             float64
CustomerID            float64
Country                object
dtype: object

In [12]:
data.isnull().sum() #Looking for NULL Values

InvoiceNo           0
StockCode           0
Description      4382
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     243007
Country             0
dtype: int64

As we can see here there are no `NULL` values for the **InvoiceNo** column.

In [13]:
data = data[data['Quantity'] > 0] 
data.shape #Check point to verify the size of the filtered dataset

(1044420, 8)

### Question 1.2: Filter the data by only transactions that happened in United Kingdom 

In [14]:
data = data[data['Country'] == 'United Kingdom'] 
data.shape

(961224, 8)

### Question 1.3: What are the most popular 5 items?

In [15]:
data = data.astype({'StockCode': str})
data[data['StockCode'] == '85123A'].head(3)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
49,536373,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 09:02:00,2.55,17850.0,United Kingdom
66,536375,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 09:32:00,2.55,17850.0,United Kingdom


In [16]:
items_list = data['StockCode'].unique().tolist()
total_sum_per_item = []
description_summary = []
invoice_summary = []
for i in items_list:
#     if data['StockCode'] == i:
    x = data[data['StockCode'] == i]
    y = x['Quantity'].sum()
    z = x['Description'].iloc[0]
    w  = x['InvoiceNo'].iloc[0]
    total_sum_per_item.append(y)
    description_summary.append(z)
    invoice_summary.append(w)

In [17]:
data_summary = pd.DataFrame(list(zip(invoice_summary, items_list, total_sum_per_item, description_summary)),
               columns =['InvoiceNo','StockCode', 'Total_Qty', 'Description'])

In [18]:
data_sum_sorted = data_summary.sort_values(by = 'Total_Qty', ascending = False)
data_sum_sorted.head()

,InvoiceNo,StockCode,Total_Qty,Description
1341,536615,84077,101464,WORLD WAR 2 GLIDERS ASSTD DESIGNS
0,536365,85123A,92476,WHITE HANGING HEART T-LIGHT HOLDER
121,536386,85099B,89143,JUMBO BAG RED RETROSPOT
142,536390,22197,84149,SMALL POPCORN HOLDER
3935,581483,23843,80995,"PAPER CRAFT , LITTLE BIRDIE"


We can see the 5 more popular items on this store:
- World War 2 Gliders
- White Hanging Heart T-Light Holder
- Jumbo Bag Red Retropot
- Small Popcorn Holder
- Paper Craft

### Question 1.4: Filter down the data to include transaction that contain the top 20 items

### Question 2.1: Consolidate the items into 1 transaction per row and each product one-hot encoded.

In [19]:
#Create the "basket"

In [20]:
#Check to make sure you did it right
basket

NameError: name 'basket' is not defined

### Question 2.2: Convert all the values to 1 when values are greater than 0 and 0 when values are 0 or less.

### Question 3.1: Apply [apriori](http://rasbt.github.io/mlxtend/user_guide/frequent_patterns/apriori/) algorithm to generate frequent item sets that have a support of at least 7%

### Question 3.2: Generate the association rules with their corresponding support, confidence and lift.

### Question 4: Based on the above rules, identify what would be the opportunity of promoting one of the antecendents.

### Question 5: Create a new text cell in your Notebook: Complete a 50-100 word summary (or short description of your thinking in applying this week's learning to the solution) of your experience in this assignment. Include: 
- What was your incoming experience with this model, if any? 
- What steps you took, what obstacles you encountered?
- How you link this exercise to real-world, machine learning problem-solving?
- What steps were missing? What else do you need to learn?